# 2024 NH 투자증권 빅데이터 경진대회

#### 종합코드

## 1. 데이터 불러오기

### 필수 라이브러리

이윤서 코드 (기준 6개)

1. 6번 파일(8월 27일 기준 ETF의 구성 종목)에서 시가총액 큰 것 순서대로 etf 추출하기

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf

In [6]:
etf_holdings = pd.read_csv('data_yoonseo/NH_CONTEST_DATA_ETF_HOLDINGS.csv', encoding='cp949')
nw_fc_stk_iem_ifo = pd.read_csv('data_chanwoo/NH_CONTEST_NW_FC_STK_IEM_IFO.csv', encoding='cp949')
stk_dt_qut = pd.read_csv('data_chanwoo//NH_CONTEST_STK_DT_QUT.csv', encoding='cp949')
historical_dividend = pd.read_csv('data_yoonseo/NH_CONTEST_DATA_HISTORICAL_DIVIDEND.csv', encoding='cp949')

In [7]:
etf_holdings_result = etf_holdings.groupby('etf_tck_cd')['mkt_vlu'].sum().reset_index()
ETF_top = etf_holdings_result.sort_values(by='mkt_vlu', ascending=False)
display(ETF_top)

,etf_tck_cd,mkt_vlu
107,VTI,1632609396735
105,VOO,1168848786417
83,SPY,537149441711
40,IVV,486185746208
67,QQQ,270418248248
...,...,...
121,YMAX,48564080
34,HDRO,18093000
95,TSL,10395509
3,AMDL,9788048


2. 1번 파일(8월 27일 기준 해외종목 정보)에서 업종분류명, 섹터분류명, 산업명 열 빼와서 6번 파일(8월 27일 기준 ETF의 구성 종목)과 병합하기

In [8]:
nw_fc_stk_iem_ifo_selected = nw_fc_stk_iem_ifo[['tck_iem_cd', 'btp_cfc_nm', 'ser_cfc_nm', 'ids_nm', 'mkt_pr_tot_amt']]
etf_holdings_and_nw_fc_stk_iem_ifo = pd.merge(etf_holdings, nw_fc_stk_iem_ifo_selected, on='tck_iem_cd', how='inner')
display(etf_holdings_and_nw_fc_stk_iem_ifo)

,etf_tck_cd,tck_iem_cd,mkt_vlu,fc_sec_eng_nm,fc_sec_krl_nm,stk_qty,wht_pct,sec_tp,btp_cfc_nm,ser_cfc_nm,ids_nm,mkt_pr_tot_amt
0,AAPB,AAPL,36858666,AAPL,애플,215737,66.778,ST,Phones & Smart Phones,Technology,Consumer Electronics,3454684.01
1,AAPB,USD,18337384,US Dollars,프로셰어즈 울트라 반도체,0,33.222,EF,-,-,-,0.00
2,AMDL,USD,3257693,US Dollars,프로셰어즈 울트라 반도체,0,33.282,EF,-,-,-,0.00
3,AMDL,AMD,6530355,AMD,에이엠디,36558,66.718,ST,Semiconductors (NEC),Technology,Semiconductors,242772.28
4,CLOU,TWLO,15400502,TWILIO INC - A,트윌리오,254933,4.266,ST,Software (NEC),Technology,Software - Infrastructure,9743.59
...,...,...,...,...,...,...,...,...,...,...,...,...
18511,TAN,CSIQ,18712550,Canadian Solar Inc,캐나디안 솔라,1382020,1.897,ST,Photovoltaic Solar Systems & Equipment,Technology,Solar,895.02
18512,TAN,AY,18652217,Atlantica Sustainable Infrastructure PLC,아틀란티카 일드,843610,1.891,ST,Alternative Electric Utilities,Utilities,Utilities - Renewable,2588.02
18513,TAN,RNW,7690201,ReNew Energy Global PLC,리뉴 에너지,1296830,0.780,ST,Renewable IPPs,Utilities,Utilities - Renewable,2063.37
18514,TAN,AMPS,5794798,Altus Power Inc,알터스 파워,1833797,0.587,ST,Solar Electric Ultilities,Utilities,Utilities - Renewable,487.97


(아래 df = etf명, etf 구성 종목, 보유종목가치, 보유종목비중, 업종분류명, 섹터분류명, 산업명)
3. ETF 종목들을 섹터분류명(ser_cfc_nm), 총 12개 섹터로 분류

In [12]:
df = etf_holdings_and_nw_fc_stk_iem_ifo[['etf_tck_cd','tck_iem_cd', 'mkt_vlu', 'wht_pct','btp_cfc_nm', 'ser_cfc_nm', 'ids_nm']]
# ETF_top.etf_tck_cd.unique()
# lst= ['VTI', 'VOO', 'SPY', 'IVV', 'QQQ', 'VEA', 'VIG', 'VWO', 'TQQQ',
#        'VYM', 'XLK', 'IEMG', 'IWM', 'VNQ', 'SCHD', 'VT', 'SPLG', 'XLF',
#        'XLV', 'XLE', 'SOXL', 'DIA', 'JEPI', 'QQQM', 'DGRO', 'SPYG',
#        'SPYV', 'MGK', 'SMH', 'TMF', 'XLP', 'XLU', 'MOAT', 'JEPQ', 'GDX',
#        'DGRW', 'SOXX', 'SPXL', 'QLD', 'NOBL', 'UPRO', 'SSO', 'PFF',
#        'TECL', 'TNA', 'QYLD', 'IBB', 'PAVE', 'XBI', 'FAS', 'XLRE', 'CIBR',
#        'ITA', 'NVDL', 'ARKK', 'SPYD', 'IHI', 'TSLL', 'SPHD', 'DIVO',
#        'PGX', 'LABU', 'XYLD', 'YINN', 'SKYY', 'URA', 'BOTZ', 'DPST',
#        'USD', 'ICLN', 'AOR', 'UDOW', 'AIQ', 'COPX', 'GRID', 'URTY',
#        'ARKW', 'ARKG', 'RYLD', 'CONL', 'NVDU', 'LIT', 'TLTW', 'JETS',
#        'NAIL', 'TAN', 'ARKF', 'ARKQ', 'RPAR', 'SVOL', 'CWEB', 'SDIV',
#        'QCLN', 'DFEN', 'DIV', 'AMZU', 'SOXQ', 'METV', 'DRIV', 'WEBL',
#        'CLOU', 'NUSI', 'AAPU', 'MSFU', 'KRBN', 'GGLL', 'FBL', 'ARKX',
#        'KBWY', 'GPIQ', 'GPIX', 'KLIP', 'KORU', 'RETL', 'MSOX', 'AAPB',
#        'AWAY', 'YMAX', 'HDRO', 'TSL', 'AMDL', 'NVD']

df_array=[]
sector_result_df = pd.DataFrame()

for ticker in ETF_top.etf_tck_cd.unique():
    filtered_df = df[df['etf_tck_cd'] == ticker]

    # ser_cfc_nm으로 그룹화하고 wht_pct 합계 계산
    grouped_df = filtered_df.groupby('ser_cfc_nm')['wht_pct'].sum().reset_index()

    # ticker 열 추가
    grouped_df['etf_tck_cd'] = ticker

    # 결과를 합치기
    sector_result_df = pd.concat([sector_result_df, grouped_df], ignore_index=True)

sector_result_df

,ser_cfc_nm,wht_pct,etf_tck_cd
0,-,0.098,VTI
1,Basic Materials,2.153,VTI
2,Communication Services,8.578,VTI
3,Consumer Cyclical,10.047,VTI
4,Consumer Defensive,5.465,VTI
...,...,...,...
657,Consumer Cyclical,55.556,TSL
658,-,33.282,AMDL
659,Technology,66.718,AMDL
660,-,33.479,NVD


3. ETF 구성 종목 개수 (분산투자 정도 파악 목적, 기준3)

In [13]:
etf_count = etf_holdings.groupby('etf_tck_cd')['tck_iem_cd'].count()

etf_count_result = etf_count.reset_index(name='tck_iem_cd_count')
etf_count_result

,etf_tck_cd,tck_iem_cd_count
0,AAPB,2
1,AAPU,10
2,AIQ,85
3,AMDL,2
4,AMZU,7
...,...,...
117,XLU,33
118,XLV,63
119,XYLD,498
120,YINN,12


4. 시가총액 큰 것 순서대로 ETF들의 거래량 내림차순 정렬
2번파일의 acl_trd_qty(누적거래수량) 활용

In [14]:
etf_tck_cd_list = ETF_top['etf_tck_cd'].tolist()

stk_dt_qut['tck_iem_cd'] = stk_dt_qut['tck_iem_cd'].str.strip()
filtered_stk_dt_qut = stk_dt_qut[stk_dt_qut['tck_iem_cd'].isin(etf_tck_cd_list)]
stk_dt_qut_result = filtered_stk_dt_qut.groupby('tck_iem_cd')['acl_trd_qty'].sum().reset_index()
stk_dt_qut_result_sorted = stk_dt_qut_result.sort_values(by='acl_trd_qty', ascending=False)
stk_dt_qut_result_sorted

,tck_iem_cd,acl_trd_qty
77,SOXL,4.791392e+09
94,TQQQ,3.532010e+09
83,SPY,3.213682e+09
96,TSLL,3.014736e+09
113,XLF,2.363635e+09
...,...,...
32,GPIX,3.500320e+06
11,ARKX,3.406643e+06
34,HDRO,2.650521e+06
71,RPAR,2.594178e+06


5. 시가-종가 간 차이와 고가-저가 간 차이 사이의 괴리 파악 (주가 변동성 파악 목적)

In [16]:
filtered_stk_dt_qut = stk_dt_qut[stk_dt_qut['tck_iem_cd'].isin(stk_dt_qut_result_sorted['tck_iem_cd'])]

# gap_ong_end 열 생성 후, 시가와 종가의 차이를 열에 넣기
filtered_stk_dt_qut['gap_ong_end'] = np.where(
    filtered_stk_dt_qut['iem_ong_pr'] > filtered_stk_dt_qut['iem_end_pr'],
    filtered_stk_dt_qut['iem_ong_pr'] - filtered_stk_dt_qut['iem_end_pr'],
    filtered_stk_dt_qut['iem_end_pr'] - filtered_stk_dt_qut['iem_ong_pr']
)
# gap_hi_low 열 생성, 고가 - 저가를 열에 넣기
filtered_stk_dt_qut['gap_hi_low'] = filtered_stk_dt_qut['iem_hi_pr'] - filtered_stk_dt_qut['iem_low_pr']
filtered_stk_dt_qut

# tck_iem_cd를 기준으로 평균 계산
result_avg = filtered_stk_dt_qut.groupby('tck_iem_cd').agg(
    avg_gap_ong_end=('gap_ong_end', 'mean'),
    avg_gap_hi_low=('gap_hi_low', 'mean')
    ).reset_index()
result_avg

# avg_gap_ong_end와 avg_gap_hi_low 기준으로 내림차순 정렬
sorted_result_avg = result_avg.sort_values(by=['avg_gap_ong_end', 'avg_gap_hi_low'], ascending=False).reset_index(drop=True)
sorted_result_avg

##시가-종가 차이, 저가-고가 차이의 평균치를 합산하여 주가 변동성이 큰 순서대로 나열하기

# avg_gap_ong_end와 avg_gap_hi_low의 합을 새로운 열로 추가
sorted_result_avg['total_gap'] = sorted_result_avg['avg_gap_ong_end'] + sorted_result_avg['avg_gap_hi_low']

# 결과를 새로운 데이터 프레임으로 생성
sum_result_avg = sorted_result_avg[['tck_iem_cd', 'total_gap']]
sum_result_avg

C:\Users\1233d\AppData\Local\Temp\ipykernel_21172\2010433661.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_stk_dt_qut['gap_ong_end'] = np.where(
C:\Users\1233d\AppData\Local\Temp\ipykernel_21172\2010433661.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_stk_dt_qut['gap_hi_low'] = filtered_stk_dt_qut['iem_hi_pr'] - filtered_stk_dt_qut['iem_low_pr']


,tck_iem_cd,total_gap
0,NVDU,14.416992
1,USD,13.823870
2,SMH,11.182910
3,NAIL,11.779965
4,QQQ,10.631935
...,...,...
117,QYLD,0.228814
118,DIV,0.233584
119,HDRO,0.199435
120,RYLD,0.217886


6. 배당금의 평균치를 가장 최근의 종가로 나눠서 시가배당률을 계산한다.
-> 이유 : 투자자들이 etf를 산 시점의 가격으로 배당률이 정해지기 때문에, 즉, 매일매일 시가배당률이 바뀌기 때문에 배당금 평균치와 최근 종가 데이터 모두 크롤링을 사용하여 구하는 게 베스트일 거라고 생각함.
- 배당금 평균치는 가장 늦게 배당금을 지급하기 시작한 etf를 찾고, 그 일자를 기준으로 현재까지의 배당금 데이터를 추출. (기준 6)

In [17]:
# 내부 조인 수행
historical_dividend_with_ETF_top = pd.merge(historical_dividend, ETF_top, on='etf_tck_cd', how='inner')

required_columns = [
    'etf_tck_cd',
    'ediv_dt',
    'ddn_amt',
    'aed_stkp_ddn_amt',
    'ddn_bse_dt',
    'ddn_pym_dt',
    'pba_dt',
    'ddn_pym_fcy_cd'
]
filtered_historical_dividend_with_ETF_top = historical_dividend_with_ETF_top[required_columns]

filtered_historical_dividend_with_ETF_top

,etf_tck_cd,ediv_dt,ddn_amt,aed_stkp_ddn_amt,ddn_bse_dt,ddn_pym_dt,pba_dt,ddn_pym_fcy_cd
0,AAPB,20231227,4.30641,4.30641,20231228,20231229,20231226,Monthly
1,AAPU,20220920,0.01041,0.01041,20220921,20220927,20220919,Quarterly
2,AAPU,20230919,0.14642,0.14642,20230920,20230926,20230918,Quarterly
3,AAPU,20231221,0.27249,0.27249,20231222,20231229,20231220,Quarterly
4,AAPU,20230321,0.07802,0.07802,20230322,20230328,20230320,Quarterly
...,...,...,...,...,...,...,...,...
995,YMAX,20240717,0.65160,0.65160,20240717,20240718,20240122,Monthly
996,YMAX,20240614,0.73000,0.73000,20240614,20240617,20240122,Monthly
997,YMAX,20240215,0.53390,0.53390,20240216,20240220,20240122,Monthly
998,YMAX,20240314,0.56800,0.56800,20240315,20240318,20240122,Monthly


배당금 제공하는 ETF들의 배당금 데이터 가져오기.

In [18]:
unique_etf_tickers_df = filtered_historical_dividend_with_ETF_top[['etf_tck_cd']].drop_duplicates()
unique_etf_tickers_list = unique_etf_tickers_df['etf_tck_cd'].tolist()

# 배당금 지급 이력을 저장할 데이터프레임 리스트
dividend_records = []

# 시작 날짜
start_date = '2020-01-30'

# 각 ETF에 대해 배당금 데이터 가져오기
for etf in unique_etf_tickers_list:
    ticker = yf.Ticker(etf)
    dividends = ticker.dividends

    # 2020년 1월 30일 이후의 데이터 필터링
    filtered_dividends = dividends[dividends.index >= start_date]

    if not filtered_dividends.empty:
        filtered_dividends = filtered_dividends.reset_index()
        filtered_dividends['ETF'] = etf  # ETF 이름 추가
        dividend_records.append(filtered_dividends)

# 결과 데이터프레임 생성
result_df1 = pd.concat(dividend_records, ignore_index=True) if dividend_records else pd.DataFrame()

# 결과 출력
if not result_df1.empty:
    print(result_df1[['ETF', 'Date', 'Dividends']])
else:
    print("No dividend data found for the specified date range.")

       ETF                      Date  Dividends
0     AAPB 2023-12-27 00:00:00-05:00      4.306
1     AAPU 2022-09-20 00:00:00-04:00      0.010
2     AAPU 2022-12-20 00:00:00-05:00      0.121
3     AAPU 2023-03-21 00:00:00-04:00      0.078
4     AAPU 2023-06-21 00:00:00-04:00      0.140
...    ...                       ...        ...
2041  YMAX 2024-07-17 00:00:00-04:00      0.652
2042  YMAX 2024-08-15 00:00:00-04:00      0.591
2043  YMAX 2024-09-19 00:00:00-04:00      0.227
2044  YMAX 2024-09-26 00:00:00-04:00      0.222
2045  YMAX 2024-10-03 00:00:00-04:00      0.208

[2046 rows x 3 columns]


월배당, 분기배당, 반기배당 모두 연배당으로 환산

In [20]:
# 배당금 지급 이력을 저장할 데이터프레임 리스트
dividend_records = []

# 각 ETF에 대해 배당금 데이터 가져오기
for etf in unique_etf_tickers_list:
    ticker = yf.Ticker(etf)
    dividends = ticker.dividends

    # 2020년 1월 30일 이후의 데이터 필터링
    filtered_dividends = dividends[dividends.index >= '2020-01-30']

    if not filtered_dividends.empty:
        filtered_dividends = filtered_dividends.reset_index()
        filtered_dividends['ETF'] = etf  # ETF 이름 추가
        dividend_records.append(filtered_dividends)

# 결과 데이터프레임 생성
result_df2 = pd.concat(dividend_records, ignore_index=True) if dividend_records else pd.DataFrame()

# 배당금 계산
if not result_df2.empty:
    # 각 ETF의 연도별 배당금 합산
    annual_dividend_results = []
    for etf in result_df2['ETF'].unique():
        etf_data = result_df2[result_df2['ETF'] == etf]

        # 연도별로 그룹화하여 배당금 합산
        etf_data['Year'] = etf_data['Date'].dt.year  # 연도 열 추가
        annual_totals = etf_data.groupby('Year')['Dividends'].sum().reset_index()

        # 총 배당금 계산 (모든 연도의 합산)
        total_dividend = annual_totals['Dividends'].sum()

        # 연간 배당금 평균 계산
        average_annual_dividend = total_dividend / len(annual_totals)

        annual_dividend_results.append({
            'ETF': etf,
            'Total Dividend': total_dividend,
            'Average Annual Dividend': average_annual_dividend
        })

    # 결과 데이터프레임 생성
    result_df2 = pd.DataFrame(annual_dividend_results)

    # 결과 출력
    print(result_df2)
else:
    print("No dividend data found for the specified date range.")

C:\Users\1233d\AppData\Local\Temp\ipykernel_21172\1714416228.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etf_data['Year'] = etf_data['Date'].dt.year  # 연도 열 추가
C:\Users\1233d\AppData\Local\Temp\ipykernel_21172\1714416228.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etf_data['Year'] = etf_data['Date'].dt.year  # 연도 열 추가
C:\Users\1233d\AppData\Local\Temp\ipykernel_21172\1714416228.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

      ETF  Total Dividend  Average Annual Dividend
0    AAPB           4.306                 4.306000
1    AAPU           1.331                 0.443667
2    AWAY           0.071                 0.035500
3    CIBR           1.250                 0.250000
4    CONL           0.109                 0.109000
..    ...             ...                      ...
108   VWO           5.649                 1.129800
109   VYM          15.264                 3.052800
110  WEBL           3.051                 3.051000
111  YINN           5.894                 1.178800
112  YMAX           5.096                 5.096000

[113 rows x 3 columns]


ETF 최근 종가 데이터. (평균 배당금과 나눠서 시가 배당률 계산하려고)

In [21]:
# 종가 데이터를 저장할 데이터 프레임 생성
closing_prices = {}

# 각 ETF의 최근 종가를 가져오기
for ticker in unique_etf_tickers_list:
    stock_data = yf.Ticker(ticker)
    # 최근 종가를 가져온 후 데이터 사전에 저장
    closing_prices[ticker] = stock_data.history(period='1d')['Close'].iloc[-1]

# 결과를 데이터 프레임으로 변환
closing_prices_df = pd.DataFrame(list(closing_prices.items()), columns=['etf_tck_cd', 'recent_close'])

# 결과 출력
print(closing_prices_df)

    etf_tck_cd  recent_close
0         AAPB     28.370001
1         AAPU     35.939999
2         AWAY     20.610001
3         CIBR     59.779999
4         CONL     22.530001
..         ...           ...
108        VWO     49.060001
109        VYM    128.679993
110       WEBL     19.440001
111       YINN     51.830002
112       YMAX     17.600000

[113 rows x 2 columns]


마지막! 시가배당률 계산하기 = (평균 배당금 / 최근 종가) * 100

In [22]:
# result_df2와 closing_prices_df를 ETF 코드로 병합
merged_df = pd.merge(result_df2, closing_prices_df, left_on='ETF', right_on='etf_tck_cd', how='inner')

# Average Annual Dividend를 recent_close로 나누기
merged_df['Dividend per Price'] = merged_df['Average Annual Dividend'] / merged_df['recent_close'] * 100

# 결과 출력
print(merged_df[['ETF', 'Average Annual Dividend', 'recent_close', 'Dividend per Price']])

      ETF  Average Annual Dividend  recent_close  Dividend per Price
0    AAPB                 4.306000     28.370001           15.178004
1    AAPU                 0.443667     35.939999            1.234465
2    AWAY                 0.035500     20.610001            0.172246
3    CIBR                 0.250000     59.779999            0.418200
4    CONL                 0.109000     22.530001            0.483799
..    ...                      ...           ...                 ...
108   VWO                 1.129800     49.060001            2.302894
109   VYM                 3.052800    128.679993            2.372397
110  WEBL                 3.051000     19.440001           15.694444
111  YINN                 1.178800     51.830002            2.274358
112  YMAX                 5.096000     17.600000           28.954545

[113 rows x 4 columns]


-----------------------------------------------------------------------

### 최성현

### 크롤링

In [29]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service as EdgeService
from datetime import datetime
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import os

In [30]:
# 모든 etf 티커명 추출
tickers = etf_holdings['etf_tck_cd'].unique()

# 각 요소에서 공백을 제거
tickers = [ticker.strip() for ticker in tickers]

# 공백을 제거한 후 결과 출력
print(tickers)

# 리스트의 요소 개수 출력
print(len(tickers))

['AAPB', 'AMDL', 'CLOU', 'DFEN', 'DPST', 'GDX', 'GPIQ', 'IWM', 'LIT', 'MOAT', 'NUSI', 'PAVE', 'QYLD', 'SPY', 'TSLL', 'URTY', 'XBI', 'AAPU', 'AMZU', 'IEMG', 'KLIP', 'KORU', 'MGK', 'NVDU', 'QCLN', 'SCHD', 'SOXQ', 'VNQ', 'XLE', 'ARKW', 'DGRO', 'DRIV', 'GGLL', 'ICLN', 'ITA', 'JETS', 'KBWY', 'NAIL', 'QLD', 'QQQM', 'RETL', 'SPYD', 'SSO', 'TNA', 'URA', 'XLP', 'XLRE', 'YMAX', 'AWAY', 'CWEB', 'DIA', 'FBL', 'JEPI', 'PFF', 'SKYY', 'TLTW', 'USD', 'VT', 'VWO', 'XLK', 'XLV', 'YINN', 'ARKG', 'ARKK', 'ARKX', 'CIBR', 'JEPQ', 'KRBN', 'MSFU', 'QQQ', 'SPHD', 'SPXL', 'TQQQ', 'VEA', 'VOO', 'BOTZ', 'DIV', 'IBB', 'METV', 'NOBL', 'SMH', 'SPYG', 'SPYV', 'UDOW', 'VTI', 'ARKF', 'CONL', 'DGRW', 'DIVO', 'HDRO', 'NVDL', 'SDIV', 'SOXL', 'SOXX', 'SPLG', 'SVOL', 'TECL', 'VYM', 'XLU', 'XYLD', 'AOR', 'ARKQ', 'GRID', 'RPAR', 'UPRO', 'VIG', 'WEBL', 'GPIX', 'IVV', 'MSOX', 'PGX', 'TSL', 'XLF', 'AIQ', 'COPX', 'FAS', 'IHI', 'LABU', 'NVD', 'RYLD', 'TAN', 'TMF']
122


In [31]:
# 엣지 옵션 설정
edge_options = Options()
edge_options.add_experimental_option('excludeSwitches', ['enable-logging', 'enable-automation'])
edge_options.add_experimental_option("useAutomationExtension", False)
edge_options.add_argument('--disable-blink-features=AutomationControlled')

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0"
edge_options.add_argument('user-agent=' + user_agent)

# 엣지 웹드라이버 설정
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=edge_options)

# 결과를 저장할 CSV 파일 경로 설정
output_file = "news_results.csv"

# 기존에 저장된 결과 파일을 불러오기
def get_existing_tickers():
    if os.path.exists(output_file):
        df = pd.read_csv(output_file, encoding='utf-8')
        return df['Ticker'].unique()  # 이미 저장된 티커 목록 반환
    return []

# CSV 파일에서 ticker에 해당하는 데이터 삭제
def delete_ticker_data(ticker):
    if os.path.exists(output_file):
        df = pd.read_csv(output_file, encoding='utf-8')
        df = df[df['Ticker'] != ticker]  # 해당 티커를 제외한 데이터
        df.to_csv(output_file, index=False)  # CSV 파일 덮어쓰기
        print(f"{ticker}에 해당하는 데이터가 CSV에서 삭제되었습니다.")

# 뉴스 크롤링 함수
def crawl_news_for_ticker(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/news/"
    print(f"Loading news for {ticker} from {url}")
    try:
        driver.get(url)
    except Exception as e:
        print(f"Error loading URL for {ticker}: {e}")
        return False

    # 페이지 로딩이 완료될 때까지 대기
    wait = WebDriverWait(driver, 30)

    try:
        # 뉴스 기사 목록이 로드될 때까지 대기
        wait.until(EC.presence_of_element_located((By.XPATH, '//li[@class="stream-item story-item yf-1usaaz9"]')))
        print(f"News loaded for {ticker}")
    except Exception as e:
        print(f"Error loading news for {ticker}: {e}")
        return False

    # 뉴스 항목 추출
    try:
        news_items = driver.find_elements(By.XPATH, '//li[@class="stream-item story-item yf-1usaaz9"]')
        print(f"Ticker: {ticker} - 뉴스 제목 및 내용:")

        for item in news_items:
            # 뉴스 제목 추출
            title_element = item.find_element(By.XPATH, './/h3[@class="clamp  yf-1sxfjua"]')
            title = title_element.text
            print(f"\n- 제목: {title}")

            # 뉴스 기사 링크 추출
            link_element = item.find_element(By.XPATH, './/div[@class="content yf-1sxfjua"]//a[@class="subtle-link fin-size-small titles noUnderline yf-1e4diqp"]')
            news_link = link_element.get_attribute('href')
            print(f"링크: {news_link}")

            # 뉴스 기사 본문 및 작성일 크롤링
            driver.execute_script(f"window.open('{news_link}', '_blank');")
            driver.switch_to.window(driver.window_handles[1])

            # 새 페이지가 로드될 때까지 대기
            wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="body yf-5ef8bf"]')))
            content = driver.find_element(By.XPATH, '//div[@class="body yf-5ef8bf"]').text
            print(f"내용: {content[:500]}...")

            # 작성일 크롤링 및 datetime 형식 변환
            try:
                date_element = wait.until(EC.presence_of_element_located((By.XPATH, '//time[@class="byline-attr-meta-time"]')))
                date_raw = date_element.get_attribute('datetime')  # 원본 날짜 값 가져오기
                date = datetime.strptime(date_raw, '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y-%m-%d')  # datetime 형식 변환
                print(f"작성일: {date}")
            except Exception as e:
                date = "작성일을 가져오지 못했습니다."
                print(f"작성일 오류: {e}")

            # 결과를 CSV 파일에 저장
            with open(output_file, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([ticker, date, title, news_link, content])

            driver.close()
            driver.switch_to.window(driver.window_handles[0])  # 원래 탭으로 돌아옴

    except Exception as e:
        print(f"Error retrieving news for {ticker}: {e}")
        return False

    return True

# 이미 크롤링된 티커 불러오기
existing_tickers = get_existing_tickers()

# 크롤링할 티커 목록에서 이미 저장된 티커는 제외
remaining_tickers = [ticker for ticker in tickers if ticker not in existing_tickers]

# 각 티커에 대해 뉴스 제목 및 내용 크롤링
for ticker in tqdm(remaining_tickers, desc="Tickers Processed"):
    success = crawl_news_for_ticker(ticker)
    if not success:
        print(f"Error occurred for ticker {ticker}, stopping and deleting its data.")
        delete_ticker_data(ticker)
        break
print("Done!")


# 브라우저 종료
driver.quit()

Tickers Processed: 0it [00:00, ?it/s]


Done!


### 박찬우

In [39]:
from sklearn.preprocessing import MinMaxScaler

In [40]:
# 스케일러 객체 생성
scaler = MinMaxScaler()

# 스케일을 적용할 컬럼 선택
columns_to_scale = ['Total Dividend','Average Annual Dividend','recent_close','Dividend per Price']

# 선택한 컬럼에 MinMaxScaler 적용
merged_df[columns_to_scale] = scaler.fit_transform(merged_df[columns_to_scale])

In [41]:
merged_df['score'] = merged_df['Total Dividend'] + merged_df['Average Annual Dividend'] + merged_df['recent_close'] + merged_df['Dividend per Price']

In [42]:
merged_df

,ETF,Total Dividend,Average Annual Dividend,recent_close,Dividend per Price,score
0,AAPB,0.138811,0.537434,0.046597,0.081675,0.804517
1,AAPU,0.042504,0.054365,0.059784,0.006493,0.163145
2,AWAY,0.001716,0.003314,0.033080,0.000765,0.038875
3,CIBR,0.039882,0.030142,0.101312,0.002092,0.173428
4,CONL,0.002946,0.012507,0.036424,0.002445,0.054322
...,...,...,...,...,...,...
108,VWO,0.182286,0.140181,0.082638,0.012254,0.417358
109,VYM,0.493542,0.380694,0.221332,0.012628,1.108196
110,WEBL,0.098184,0.380469,0.031042,0.084459,0.594154
111,YINN,0.190217,0.146309,0.087463,0.012100,0.436089
